Importo le librerie

In [48]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import precision_score,accuracy_score

## Caricamento del dataset

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/elabcn/water_potability.csv')

df.info()

test_df = pd.read_csv('/content/drive/MyDrive/elabcn/test.csv')

## Preprocessing dei dati

In [50]:
df['ph'].fillna(value=df['ph'].median(),inplace=True) #inserisco le medie delle colonne dove ci sono i campioni nulli
df['Sulfate'].fillna(value=df['Sulfate'].median(),inplace=True)
df['Trihalomethanes'].fillna(value=df['Trihalomethanes'].median(),inplace=True)

test_df['ph'].fillna(value=df['ph'].median(),inplace=True) #inserisco le medie delle colonne dove ci sono i campioni nulli
test_df['Sulfate'].fillna(value=df['Sulfate'].median(),inplace=True)
test_df['Trihalomethanes'].fillna(value=df['Trihalomethanes'].median(),inplace=True)


In [51]:
X = df.drop('Potability',axis=1).values #elimino la colonna sulla potabilità dal dataset e la inserisco come etichetta
y = df['Potability'].values

In [52]:
df

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,7.036752,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.057858,6.635246,333.073546,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.541732,9.275884,333.073546,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0
...,...,...,...,...,...,...,...,...,...,...
3267,4.668102,193.681735,47580.991603,7.166639,359.948574,526.424171,13.894419,66.687695,4.435821,1
3268,7.808856,193.553212,17329.802160,8.061362,333.073546,392.449580,19.903225,66.610937,2.798243,1
3269,9.419510,175.762646,33155.578218,7.350233,333.073546,432.044783,11.039070,69.845400,3.298875,1
3270,5.126763,230.603758,11983.869376,6.303357,333.073546,402.883113,11.168946,77.488213,4.708658,1


Split del dataset e rescaling


In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101) #split del dataset in 

scaler = StandardScaler()   #preprocessing dei dati: rescaling e centering rispetto alla media ed alla dev. std
scaler.fit(X_train)         #calcolo media e dev std
X_train = scaler.transform(X_train) 
X_test = scaler.transform(X_test)


test_case = test_df.drop('Potability',axis=1).values
test_case = scaler.transform(test_case)

In [54]:
test_case

array([[-3.24414660e-01,  2.34460807e+00,  1.69803265e+00,
         6.20745626e-01,  2.26706589e+00, -9.35155816e-01,
         2.47392786e-01,  7.97432344e-01,  2.81514469e-01],
       [-2.05927420e-01,  1.07738248e+00, -4.01350411e-01,
         1.02149677e+00,  2.45363273e-01, -5.16492050e-01,
        -1.33329970e-01,  8.00026177e-01,  1.52500906e+00],
       [ 1.63358897e+00, -1.51810257e+00, -1.00619117e+00,
         1.45028027e+00, -6.33437124e-01,  2.06665213e+00,
        -1.75456652e+00,  6.91975375e-01, -1.40930658e-01],
       [ 1.34618176e+00, -2.05596275e+00, -2.52288255e-01,
         5.59417704e-01, -9.20270376e-01,  3.05317734e-01,
         1.37374038e-01,  4.53982600e-01,  1.50523952e-03]])

# Training sui classificatori

In [55]:
modelli =[("SVC", SVC()),
        ("SGDC", SGDClassifier())]

risultati = []
names = []
finalResults = []

for name,model in modelli:
    model.fit(X_train, y_train)
    model_results = model.predict(X_test)
    score = precision_score(y_test, model_results,average='macro')
    risultati.append(score)
    names.append(name)
    finalResults.append((name,score))
    
finalResults.sort(key=lambda k:k[1],reverse=True)

In [56]:
finalResults

[('SVC', 0.6840374331550803), ('SGDC', 0.5008715923887932)]

# Esempi di predizioni


In [57]:
res = []
modello_utilizzato = modelli[1][1]
for test in test_case:
  res.append(modello_utilizzato.predict([test])[0])

test_df.assign(Prediction = res)



,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability,Prediction
0,6.581878,272.982745,37169.444404,8.114731,416.083481,351.476839,15.129334,79.261026,4.201663,0,0
1,6.755146,231.260131,18536.698647,8.757133,342.548014,385.114648,13.888834,79.302436,5.162730,0,0
2,9.445130,145.805402,13168.529156,9.444471,310.583374,592.659021,8.606397,77.577460,3.875165,1,1
3,9.024845,128.096691,19859.676476,8.016423,300.150377,451.143481,14.770863,73.778026,3.985251,1,1
